In [1]:
import libs.utils
import libs.vars
import xarray

In [2]:
ensemble = libs.vars.ensemble()

areacello_kwargs = {
    'experiment_id': 'piControl',
    'variable_id': 'areacello',
    'variant_label': None
}

si_kwargs = {
    'experiment_id': 'ssp585',
    'frequency': 'mon',
    'variable_id': 'siconc'
}

In [3]:
for i, item in enumerate(ensemble):
    kwargs = {}
    kwargs['experiment_id'] = item['experiment_id']
    kwargs['source_id'] = item['source_id']
    kwargs['variant_label'] = item['variant_label']
    
    for k in si_kwargs:
        kwargs[k] = si_kwargs[k]

    kwargs['process_files'] = True
    kwargs['save_to_local'] = True

    ensemble[i]['file'] = libs.utils.download_variable(**kwargs)
    print('-' * 20)

Requesting:
-> https://esgf-index1.ceda.ac.uk/esg-search/search/?experiment_id=ssp585&format=application%2Fsolr%2Bjson&grid_label=gn&latest=true&limit=20&mip_era=CMIP6&offset=0&replica=false&source_id=UKESM1-0-LL&type=Dataset&variable_id=siconc&frequency=mon&variant_label=r2i1p1f2
Results:
-> CMIP6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.r2i1p1f2.SImon.siconc.gn.v20200420|esgf-data3.ceda.ac.uk
   -> Already exists, skipping: _data/cmip6/UKESM1-0-LL/siconc_SImon_UKESM1-0-LL_ssp585_r2i1p1f2_gn_201501-204912.nc
   -> Already exists, skipping: _data/cmip6/UKESM1-0-LL/siconc_SImon_UKESM1-0-LL_ssp585_r2i1p1f2_gn_205001-210012.nc
-> Processing:
   -> Merged
   -> Set time encoding
   -> Processed file already exists, skipping write
--------------------
Requesting:
-> https://esgf-index1.ceda.ac.uk/esg-search/search/?experiment_id=ssp585&format=application%2Fsolr%2Bjson&grid_label=gn&latest=true&limit=20&mip_era=CMIP6&offset=0&replica=false&source_id=NorESM2-LM&type=Dataset&variable_id=siconc&freq

In [ ]:
# TODO: fix failing ACCESS-CM2 and NorESM regridding
filename = f'siconc_SImon_UKESM1-0-LL_ssp585_r2i1p1f2_gn_201501-204912.nc'
local_file = f'_data/cmip6/UKESM1-0-LL/{filename}'
ukesm_siconc = xarray.open_mfdataset(
    paths=local_file, 
    combine='by_coords',
    use_cftime=True
)
regrid_kwargs = {
   'grid': ukesm_siconc, 
   'method': 'nearest_s2d',
}

for item in ensemble[2:]:
    item_data = xarray.open_mfdataset(
        paths=item['file'], 
        combine='by_coords',
        use_cftime=True
    )
    save_file = str(item['file']).replace('.nc', '_regrid.nc')
    print(f'Saving to:')
    print(f'-> {save_file}')
    data_regridded = libs.utils.regrid(item_data, save_file=save_file, **regrid_kwargs)
    item_data.close()
    data_regridded.close()


Saving to:
-> _data/cmip6/CanESM5/siconc_SImon_CanESM5_ssp585_r1i1p2f1_gn_201501-210012_processed_regrid.nc
[########################################] | 100% Completed |  8.5s
Saving to:
-> _data/cmip6/CMCC-ESM2/siconc_SImon_CMCC-ESM2_ssp585_r1i1p1f1_gn_201501-210012_processed_regrid.nc
[########################################] | 100% Completed |  7.2s
Saving to:
-> _data/cmip6/ACCESS-CM2/siconc_SImon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012_processed_regrid.nc
[#############                           ] | 33% Completed | 35.4s